## Initial problem

We are working with a solution to
$$
\begin{equation}
\begin{cases}
      c\frac{\partial P}{\partial t} + \vec{\nabla}\vec{q} = f\text{ in }\Omega \\
      \vec{q} = -a\vec\nabla P
\end{cases}
\label{eq:cfd_problem} \tag{1}
\end{equation}
$$
with Dirichlet ($P = P_e$ on $\partial\Omega_D$) and Neumann ($-a\frac{\partial P}{\partial \vec\eta} = g$ on $\partial\Omega_N$) boundary conditions

For this particular problem, we focus on uniform and constant $a$ and $c$ and $f = 0$.

We want to calculate sensitivity $\frac{dg}{da}$ of a function
$\begin{equation}
g = \frac{1}{\left|\Omega\right|}\int\limits_\Omega Pdx
\tag{2}
\end{equation}$

If we subdivide $\Omega$ into elements (triangles), $P$ becomes a vector with elements $P_i$ corresponding to an average value of $P$ over $i$-th triangle.

Then, we could approximate $g$ by:
$\begin{equation}
g = \frac{1}{N}\sum\limits_{i = 1}^{N} P_i = \frac{\left(\vec{1}, \vec{P}\right)}{N}
\tag{3}
\end{equation}$
where $\vec{1} = \left( \begin{matrix} 1\\ 1\\ 1\\ ... \end{matrix}\right)$ &mdash; an $N$-vector of ones.

## LMHFE

LMHFE gives a solution as an average value of $P$ over element edges $\vec{P_T}$ ($P_{T,j}$ &mdash; average $P$ over $j$-th mesh edge) as a solution to a system of linear algebraic equations:
$\begin{equation}
M\vec{P}^{n + 1}_T = \vec b(\vec{P}^n_T)
\tag{4}
\end{equation}$
For time step $n + 1$, it is then coverted back to $\vec P$ using:
$$P_i^{n + 1} = \frac{a}{\beta}\sum\limits_{\text{edge }j \in\text{ boundary of triangle }i} \alpha_j P_{T,j}^{n+1} + \frac{\lambda_A}\beta P^n_i + \frac{Q_S^i}{\beta}$$
Here, $Q_S^i = 0$ as it's proportional to a mean value of $f = 0$ over $i$-th triangle.

$\lambda_A = \frac{c\|A\|}{\Delta t} = \lambda$ - is a scalar in case of triangular grid constructed of triangles of equal area.

The previous equation could be written as:
$$ \vec{P}^{n+1} = U\vec{P}_T^{n+1} + \frac\lambda\beta \vec{P}^n $$
or
$\begin{equation}
\vec P^{n+1} = \frac{a}{\beta}U_0\vec P_T^{n+1} + \frac\lambda\beta \vec P^n
\tag{5}
\end{equation}$
where $U_0$ depends only on mesh geometry and is not a funcion of problem parameters.

## Sensitivity calculation

Now, let's evaluate sensitivity:

$$ \frac{dg}{da} = \frac{\partial g}{\partial a} + \frac{\partial g}{\partial \vec P} \frac{d \vec P}{d a} $$

Since $g$ has no explicit dependence on $a$, only the second term here is non-zero.
From now on, let's denote partial derivatives with partial symbol and an index, e.g. $\partial g_\vec P = \frac{\partial g}{\partial \vec P}$.
Also, let's denote full derivatives with index only, e.g. $\vec P_a = \frac{d \vec P}{da}$.

$$ \frac{dg^n}{da} = \partial g^n_\vec P \vec P^n_a $$

$$ \frac{dg^n}{da} = \partial g^n_\vec P \left(\left(U \vec P_T^n\right)_a + \frac\lambda\beta\vec P^{n-1}_a\right) $$
$$ \frac{dg^n}{da} = \partial g^n_\vec P \left(\left(U \vec P_T^n\right)_a + \frac\lambda\beta \left(\left(U \vec P_T^{n-1}\right)_a + \frac\lambda\beta\vec P^{n-2}_a \right)\right) $$
We keep expanding $\vec P^k_a$ until $k = 0$. At $k = 0$ $\vec P^k$ is inital condition and does not depend on $a$. Therefore: $\vec P^0_a = 0$ and expression for $\frac{dg}{da}$ becomes:
$\begin{equation}
\frac{dg^n}{da} = \partial g^n_\vec P \sum\limits_{k = 0}^{n - 1} \left( \frac\lambda\beta \right)^k \left(U \vec P_T^{n - k} \right)_a
\tag{6}
\end{equation}$

Let's evaluate $\partial g^n_\vec P$:

$\begin{equation}
\partial g^n_\vec P = \frac{1}{N}\partial \left( \vec 1, \vec P \right)_\vec P = \frac{1}{N}\vec 1^T
\tag{7}
\end{equation}$

and $\left( U\vec P^k_T \right)_a$:
$\begin{equation}
\left(U\vec P_T^k\right)_a = \left(\frac{a}{\beta}U_0\vec P_T^k\right)_a = \frac{1}{\beta}U_0\vec P_T^k + \frac{a}{\beta}U_0\left(\vec P_T^k\right)_a
\tag{8}
\end{equation}$

$\vec P^k_T$ is given by
$$ M\vec P^k_T = \vec b \left(\vec P^{k-1}_T\right) = \vec b^k$$
$$M_a\vec P^k_T + M \left( \vec P^k_T \right)_a = \left( \vec b^k \right)_a$$
$$\left( \vec P^k_T \right)_a = M^{-1}\left( \left( \vec b^k \right)_a - M_a \vec P^k_T \right)$$

Since $\vec b^k = \vec b^k (\vec P^{k-1}_T) \Rightarrow \left(\vec b^k\right)_a = \partial \vec b^k_a + \partial \vec b^k_{\vec P^{k-1}_T} \left( \vec P^{k-1}_T \right)_a$, we get a recursive formula for $\left( \vec P^k_T \right)_a$:

$\begin{equation}
\left( \vec P^k_T \right)_a = M^{-1} \left( \partial \vec b^k_a + \partial \vec b^k_{\vec P^{k-1}_T} \left( \vec P^{k-1}_T \right)_a - M_a \vec P^k_T \right)
\tag{9.1}
\end{equation}$

Therefore, each step we need to evaluate **(7)** from $\left( \vec P^k_T \right)_a$ **(8)** and save the latter to calculate $\left( \vec P^{k+1}_T \right)_a$ on the next step.
Initial condition $\vec P^0_T$ does not depend on problem parameters leading to
$\begin{equation}
\left( \vec P^0_T \right)_a = 0
\tag{9.2}
\end{equation}$.

Let's place **(7)** and **(8)** into **(6)**, and also write a recursive formula for calculating **(5)**:

$$\frac{dg^n}{da} = \partial g_{\vec P} \sum\limits_{k=0}^{n-1}\left( \frac{\lambda}{\beta}\right)^k\left(U\vec P^{n-k}_T\right)_a = \partial g_{\vec P} \sum\limits_{k=1}^{n-1}\left(\frac{\lambda}{\beta}\right)^k\left(U\vec P^{n-k}_T\right)_a + \partial g_{\vec P} \left( U\vec P^n_T \right)_a$$

$$\frac{dg^{n-1}}{da} = \partial g_{\vec P} \sum\limits_{k=0}^{n-2}\left( \frac{\lambda}{\beta}\right)^k\left(U\vec P^{n-k-1}_T\right)_a = \partial g_{\vec P} \sum\limits_{k=1}^{n-1}\left( \frac{\lambda}{\beta}\right)^{k - 1}\left(U\vec P^{n-k}_T\right)_a$$

$$ \frac{dg^n}{da} = \frac{\lambda}{\beta} \frac{dg^{n-1}}{da} + \partial g_{\vec P} \left(U\vec P^n_T\right)_a$$

$$\frac{dg^n}{da} = \frac{\lambda}{\beta}\frac{dg^{n-1}}{da} + \partial g_{\vec P} \left( \frac{1}{\beta}U_0\vec P^n_T + \frac{a}{\beta}U_0 \left(\vec P^n_T\right)_a \right)$$

$\begin{equation}
\frac{dg^n}{da} = \frac{\lambda}{\beta}\frac{dg^{n-1}}{da} + \frac{1}{\beta}\partial g_{\vec P} U_0 \vec P^n_T + \frac{a}{\beta} \partial g_{\vec P} U_0 \left(\vec P^n_T \right)_a
\tag{10}
\end{equation}$

Formula **(10)** is valid not only in case of **(3)**, but in a more general case of known and constant $\partial g_{\vec P}$.
Time-step-dependant $\partial g_{\vec P^n}$ will require a slight modification to a formula.

Direct expansion of **(5)** leads to:
$\begin{equation}
\frac{dg^0}{da} = 0
\tag{11}
\end{equation}$

## $M$ and $\vec b$ in LMHFE

*$\alpha$, $\alpha_i$, $B_{ij}, \beta, \lambda, n_f$ depend only on geometry and act as constants when differentiating by problem parameters.*

If $M = \left( m_{ij} \right)$ then its diagonal terms are:

$$ m_{ii} = m^A_{ii} + m^B_{ii}, m^A_{ii} = a\left(B_{ii}^{-1} - \frac{\alpha_i^2}{\alpha}\right) + \frac{1}{n_f} \frac{c\|A\|}{\Delta t}$$

where $A$ and $B$ are triangles adjacent to the $i$-th edge, and its off-diagonal terms are:

$$ m_{ij} = a \left( B^{-1}_{ij} - \frac{\alpha_i\alpha_j}{\alpha} \right) $$

Therefore, the full derivative $M_a$ has terms:

$$\begin{equation}
\left(m_{ii}\right)_a = 2 \left(B^{-1}_{ii} - \frac{\alpha_i^2}{\alpha}\right)
\tag{12.1}
\end{equation}$$

$$\begin{equation}
\left(m_{ij}\right)_a = B^{-1}_{ij} - \frac{\alpha_i\alpha_j}{\alpha}
\tag{12.2}
\end{equation}$$

Notice that $M$ does not change from step to step as long as $a$ and $c$ are constant and time step does not vary.

For $\vec b$:

$$ \vec b_i^k = \frac{c \left(\|A\| + \|B\|\right) \vec P_{Ti}^{k-1}}{n_f \Delta t} $$

where $A$ and $B$ are triangles adjacent to $i$-th edge, and partials:

$$\begin{equation}
\partial \vec b_a = 0
\tag{13.1}
\end{equation}
$$
and
$$\begin{equation}
\left( \partial \vec b_{\vec P_T^{k-1}} \right)_{ii} = \frac{c\left(\|A\| + \|B\|\right)}{n_f \Delta t}
\tag{13.2.1}
\end{equation}$$
$$\begin{equation}
\left( \partial \vec b_{\vec P_T^{k-1}} \right)_{ij} = 0
\tag{13.2.2}
\end{equation}$$
$\partial \vec b_{\vec P^{k-1}_T}$ is a diagonal matrix.

With this, we could rewrite **(9.1)** as:

$$\begin{equation}
M\left(\vec P^k_T \right)_a = \partial \vec b^k_{\vec P_T^{k-1}} \vec P^{k-1}_T - M_a \vec P^k_T
\tag{14}
\end{equation}$$

To save time from inverting $M$ and performing extra matrix multiplications, we will find $\left( \vec P_T^k \right)_a$ as a sulution to **(14)** rather than calculating **(9.1)** directly.

## Computation algorithm for sensitivity $\frac{dg^n}{da}$ on the $n$-th step of LMHFE

Initial conditions (step $n=0$): ones of problem **(1)** with addition of **(9.2)** and **(11)**.

Then, for the $n$-th step:
1. Perform an LMHFE solution step.
   Since neither equations **(10)**, **(9.1)** nor next LMHFE step explicitly include solution over triangles $\vec P^n$, obtaining it back from mean edge values $\vec P_T^n$ via **(5)** is optional. 
1. Using **(12.1-2)** and **(13.2)** compute $M_a$ and $\partial \vec b_{\vec P_T^{k-1}}$ (optionally, this could be skipped and performed on-demand when calculating right-hand part of **(14)**). With constant $a$ and $c$ and a uniform time step it is possible to calculate $M$ only once per solution.
1. Find $\left( \vec P^n_T \right)_a$ using **(14)** or **by adjoints**.
1. Calculate $\frac{dg^n}{da}$ using **(10)**

## Gradient by adjoints

Let's expand **(9.1)** with **(13.1)** up to $\left( \vec P_T^{0} \right)_a = 0$ (denote $V^k = M^{-1}\partial \vec b_{\vec P_T^{k-1}}$:

$$ \left( \vec P_T^k \right)_a = V^k \left( V^{k-1} \left( V^{k-2} \left( ... V^2\left( -M^{-1}M_a \vec P_T^1 \right) ... \right) - M^{-1}M_a \vec P_T^{k-2} \right) - M^{-1}M_a \vec P^{k-1}_T \right) - M^{-1}M_a \vec P_T^k $$

*Do not confuse $V^2$ with V __squared__*

By transposing the equation we get:

$$\left(\vec P_T^k \right)_a^T = \left( ... \left(-(\vec P^1_T)^T(M_a)^T(M^{-1})^T\right)(V^2)^T ... \right) (V^k)^T - (\vec P^k_T)^T(M_a)^T(M^{-1})^T $$